<a href="https://colab.research.google.com/github/21-Srivathsa/My-projects/blob/main/t20i_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Historic data/Ball by ball data/Ipl/1254060.csv')

In [ ]:
data.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1254060,2021,2021-04-11,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Kolkata Knight Riders,Sunrisers Hyderabad,N Rana,Shubman Gill,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1254060,2021,2021-04-11,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.2,Kolkata Knight Riders,Sunrisers Hyderabad,N Rana,Shubman Gill,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# FUNCTIONS

In [ ]:
def ispowerplay(data):
  arr = []
  for i in range(len(data["match_id"])):
    if data["ball"][i]<6.1 and data["innings"][i] == 1:
      arr.append(1)
    else:
      arr.append(0)
  data["Powerplay"] = arr
  

def isdeath(data):
  arr = []
  for i in range(len(data["match_id"])):
    if data["ball"][i] > 15 and data["innings"][i] == 1:
      arr.append(1)
    else:
      arr.append(0)
  data["Death"] = arr
  

In [ ]:
ispowerplay(data)
isdeath(data)


In [ ]:
data.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'Powerplay', 'Death'],
      dtype='object')

In [ ]:
def last3overruns(data):
  arr = []
  for i in range(len(data["match_id"])):
    pre = data["ball"][i]
    if pre < 3 and data["innings"][i] == 1:
      runs = 0
      for j in range(i):
        runs+=(data["runs_off_bat"][j] + data["extras"][j])
      arr.append(runs)
    else:
      prev1 = pre - 3
      
      runs = 0

      for k in range(len(data["match_id"])):
        if prev1 < data["ball"][k] < pre and data["innings"][k] == 1:
          runs +=(data["runs_off_bat"][k] + data["extras"][k])
      arr.append(runs)
      
      
  data["Last3overruns"] = arr


In [ ]:
last3overruns(data)

In [ ]:
def wicket_in_last_three_overs(x):
    arr = ['lbw','caught','bowled','run out','caught and bowled','stumped']
    c = 0
    for i in range(len(data['ball'])):
        if x-3 <= data['ball'][i] < x and data['innings'][i]==1:
            if data['wicket_type'][i] in arr:
                c = c+1
    return(c)
def wickets3overs(data):
  lst = []
  for i in data['ball']:
      lst.append(wicket_in_last_three_overs(i))
  data["Last3overwkts"] = lst


In [ ]:
wickets3overs(data)

In [ ]:
def non_striker(data):
  arr = []
  for i in range(len(data["match_id"])):
    if data["innings"][i] == 1:
      non = data["non_striker"][i]
      runs = 0
      for k in range(i):
        if data["striker"][k] == non:
          runs+=data["runs_off_bat"][k]
      arr.append(runs)
    else:
      arr.append(0)
  data["runs_nonstriker"] = arr
      

In [ ]:
non_striker(data)

In [ ]:
def striker(data):
  arr = []
  for i in range(len(data["match_id"])):
    if data["innings"][i] == 1:
      on = data["striker"][i]
      runs = 0
      for k in range(i):
        if data["striker"][k] == on:
          runs+=data["runs_off_bat"][k]
      arr.append(runs)
    else:
      arr.append(0)
  data["runs_striker"] = arr
  

In [ ]:
striker(data)

In [ ]:
if np.isnan(data["wides"][0]) == True:
  print("ABD")

ABD


In [ ]:
def crr(data):
  arr = []
  tb = 0
  runs = 0
  for i in range(len(data["match_id"])):
    if data["innings"][i] == 1:
      runs += (data["runs_off_bat"][i] + data["extras"][i])
      tb+=1
      if np.isnan(data["wides"][i]) == False or np.isnan(data["noballs"][i]) == False:
        tb -= 1
    rr = 6*runs/tb
    arr.append(rr)
  data["CRR"] = arr
      
      
    

In [ ]:
crr(data)

In [ ]:
data["CRR"][96]

9.8125

In [ ]:
def total_runs(data):
    c = 0
    for i in range(len(data['ball'])):
        if data['innings'][i]==1:
            c += data['runs_off_bat'][i]+data['extras'][i]
    arr = []
    for i in range(len(data['ball'])):
        if data['innings'][i] == 1:
            arr.append(c)
        else:
            arr.append(0)
    data["Total_Runs"] = arr

In [ ]:
total_runs(data)

In [ ]:
def wickets_till(data):
  w = 0
  arr = []
  for i in range(len(data['ball'])):
        if data['innings'][i]==1:
            if data["wicket_type"][i] is not np.nan:
              w+=1
        arr.append(w)
  data["Wicket_till"] = arr

In [ ]:
wickets_till(data)

# VERSION 2.0


In [ ]:
def sr_striker(data):
  arr = []
  
  for i in range(len(data["match_id"])):
    if data["innings"][i] == 1:
      on = data["striker"][i]
      runs = 0
      count = 0
      for k in range(i):
        if data["striker"][k] == on:
          runs+=data["runs_off_bat"][k]
          count+=1
          if np.isnan(data["wides"][k]) == False:
            count-=1
      if count == 0:
        sr = 0
      else:
        sr = 100*runs/count
      arr.append(sr)
    else:
      arr.append(0)
  data["sr_striker"] = arr

In [ ]:
sr_striker(data)

In [ ]:
def sr_nonstriker(data):
  arr = []
  
  for i in range(len(data["match_id"])):
    if data["innings"][i] == 1:
      on = data["non_striker"][i]
      runs = 0
      count = 0
      for k in range(i):
        if data["striker"][k] == on:
          runs+=data["runs_off_bat"][k]
          count+=1
          if np.isnan(data["wides"][k]) == False:
            count-=1
      if count == 0:
        sr = 0
      else:
        sr = 100*runs/count
      arr.append(sr)
    else:
      arr.append(0)
  data["sr_nonstriker"] = arr

In [ ]:
sr_nonstriker(data)

In [ ]:
data.to_csv("/content/drive/MyDrive/Historic data/checkp1.csv")

# FUNCTIONS PARI SAMAPTHAM

In [ ]:
def right_csv(x):
  temp = x.split(".")
  if temp[0].isdigit() is True:
    return "yes"
  else:
    return "no"

In [ ]:
path = "/content/drive/MyDrive/Historic data/Ball by ball data/t20is"
import os
files = [i for i in os.listdir("/content/drive/MyDrive/Historic data/Ball by ball data/t20is")]

In [ ]:
path2 = "/content/drive/MyDrive/Historic data/Ball by ball data/Ipl/"
files2 = [i for i in os.listdir("/content/drive/MyDrive/Historic data/Ball by ball data/Ipl/")]


In [ ]:
new = [i for i in files if right_csv(i) == "yes"]
new2 = [i for i in files2 if right_csv(i) == "yes"]


In [ ]:
path = "/content/drive/MyDrive/Historic data/Ball by ball data/t20is/"
path_new = []
for i in new:
  path_new.append(path + i)

In [ ]:
path = "/content/drive/MyDrive/Historic data/Ball by ball data/Ipl/"
path_new2 = []
for i in new2:
  path_new2.append(path + i)

In [ ]:
path_new1 = path_new + path_new2

In [ ]:
frames = []
for i in path_new1:
  data = pd.read_csv(i)
  ispowerplay(data)
  isdeath(data)
  striker(data)
  non_striker(data)
  crr(data)
  last3overruns(data)
  wickets3overs(data)
  total_runs(data)
  sr_striker(data)
  sr_nonstriker(data)
  wickets_till(data)
  frames.append(data)
result = pd.concat(frames)
result.to_csv('/content/drive/MyDrive/Historic data/merged_t20i.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()
